# Crawl data bất động sản ở đà nẵng

Link: https://batdongsan.com.vn/nha-dat-ban-da-nang/


Import thư viện.


In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import csv


Hàm tiền xử lí diện tích, bỏ đơn vị m² ra khỏi trường diện tích.


In [ ]:
def handleArea(s: str):
    try:
        if s != '':
            return s.split(' ')[0]
        return s
    except:
        return ''


Hàm tiền xử lí loại bất động sản.


In [ ]:
def handleType(s: str):
    try:
        return s.replace('Loại tin đăng: ', '')
    except:
        return ''


Hàm tiền xử lí địa chỉ, chỉ lấy địa chỉ phường/xã và quận/huyện.


In [ ]:
def handleAddress(s: str):
    result = ""
    if s.find(',') > 1:
        try:
            arr = s.split(',')
            for item in arr:
                if item.find('Phường') != -1 or item.find('Xã') != -1:
                    result += item.lstrip().rstrip() + ", "
            return result + arr[-2]
        except:
            return result
    else:
        return result


Hàm tiền xử lí giá, chuyển số tiền từ dạng chữ (100 triệu) sang dạng số (100) và xử lí giá không hợp lệ (Thỏa thuận).


In [ ]:
def handlePrice(price: str, area: str):
    result = ''
    try:
        if price != 'Thỏa thuận':
            if price.find('m²') != -1:
                result = str(float(price.split(' ')[0]) * float(area))
            elif price.find('tỷ') != -1:
                result = str(float(price.split(' ')[0]) * 1000)
            else:
                result = str(float(price.split(' ')[0]))
    except:
        pass
    return result


Hàm ghi data crawl vào csv.

In [ ]:
def write_csv(row, file_name):
    with open(file_name, mode='a', newline='', encoding='utf-8-sig') as outfile:
        writer = csv.writer(outfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(row)


Tên của các file csv đầu ra.


In [ ]:
name_csv_raw = 'raw_data.csv'
name_csv_cleaned = 'cleaned_data.csv'
write_csv(['Title', 'Address', 'Type', 'Area', 'Price'], name_csv_raw)
write_csv(['Title', 'Address', 'Type', 'Area', 'Price'], name_csv_cleaned)


Số trang data muốn cào ở base_url.

In [ ]:
num_of_page = 100
base_url = "https://batdongsan.com.vn/nha-dat-ban-da-nang/"


Thêm option để selenium crawl data chạy ngầm mà không bật lên cửa sổ trình duyệt.


In [ ]:
options = Options()
options.add_argument('--headless')
options.add_argument(
    'user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36"')


Bắt đầu crawl data.


In [ ]:
for i in range(1, num_of_page + 1):
    print('Trang số', str(i))
    driver = webdriver.Chrome(
        ChromeDriverManager().install(), options=options)
    driver.get(base_url + "p" + str(i))

    elements = driver.find_elements(
        by=By.CSS_SELECTOR, value=".js__product-link-for-product-id")
    links = [el.get_attribute('href') for el in elements]

    for link in links:
        driver_detail = webdriver.Chrome(
            ChromeDriverManager().install(), options=options)
        driver_detail.get(link)
        title_detail = driver_detail.find_element(
            By.CLASS_NAME, "pr-title").get_attribute('textContent')
        address_detail = driver_detail.find_element(
            By.CLASS_NAME, "re__pr-short-description").get_attribute('textContent')
        type_detail = driver_detail.find_element(
            By.CLASS_NAME, "re__pr-specs-product-type").get_attribute('textContent')
        area_detail = ""
        price_detail = ""

        t1 = driver_detail.find_elements(
            by=By.CSS_SELECTOR, value=".re__pr-short-info > .re__pr-short-info-item > .title")
        t2 = driver_detail.find_elements(
            by=By.CSS_SELECTOR, value=".re__pr-short-info > .re__pr-short-info-item > .value")
        temp1 = [el.text for el in t1]
        temp2 = [el.text for el in t2]
        for _ in range(len(temp1)):
            if temp1[_] == 'Mức giá':
                price_detail = temp2[_]
            elif temp1[_] == 'Diện tích':
                area_detail = temp2[_]
        write_csv([title_detail, address_detail,
                   type_detail, area_detail, price_detail], name_csv_raw)

        area_detail = handleArea(area_detail)
        price_detail = handlePrice(price_detail, area_detail)
        write_csv([title_detail, handleAddress(address_detail),
                   handleType(type_detail), area_detail, price_detail], name_csv_cleaned)
        driver_detail.close()

    driver.close()
